In [1]:
from pprint import pprint

from dotenv import load_dotenv
from langchain_community.chat_models import ChatTongyi
from langchain_core.messages import HumanMessage

import overview.config as cfg
from overview.agent import main_agent, search_subagent
from overview.Analysis.PaperMeta import PaperMetaAnalyzer
from overview.tools.VectorStore import VectorStore

load_dotenv()


True

In [2]:
vec_store = VectorStore(
    md_dir="./examples/Example/md",
    store_dir="./examples/Example/vectStore",
)

paper_info = PaperMetaAnalyzer(
    file_paths=["examples/Example/pdf/1706.03762v7.pdf"]
).metadata


sync_store completed
📄 正在分析: 1706.03762v7.pdf
DONE : examples/Example/pdf/1706.03762v7.pdf



In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
sub_model = ChatTongyi(model="qwen-max")
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro")
agent = main_agent(
    model, subagents=[search_subagent(sub_model)], tools=[vec_store.retriever_tool]
)


In [5]:
from IPython.display import display, Markdown, HTML, clear_output
import time

def print_header(role, content, color):
    """打印带颜色的标题（适配黑色背景）"""
    # User 用亮绿色 (#00ff00), AI 用亮青色 (#00ffff)
    html_code = f"""
    <div style="border-left: 5px solid {color}; padding-left: 10px; margin-top: 10px;">
        <span style="color: {color}; font-weight: bold; font-size: 1.1em;">{role}</span>
        <br>
        <span style="color: #cccccc;">{content}</span>
    </div>
    """
    display(HTML(html_code))


def print_ai_markdown(content):
    """使用 Markdown 渲染 AI 的回复（支持公式和代码高亮）"""
    display(Markdown(content))
    display(HTML("<hr style='border-color: #333;'>"))  # 灰色分割线


def parse_ai_response(raw_content):
    if isinstance(raw_content, str):
        return raw_content
    elif isinstance(raw_content, list):
        # 拼接列表中所有的 'text' 字段
        text_parts = []
        for item in raw_content:
            if isinstance(item, dict) and "text" in item:
                text_parts.append(item["text"])
            elif isinstance(item, str):
                text_parts.append(item)
        return "".join(text_parts)
    else:
        return str(raw_content)


# --- 主逻辑 ---

print("🟢 交互开始 (输入 'exit' 结束)")

# 首次提问
question = input("请输入你的问题: ")

if question.strip().lower() not in ["exit", "quit", ""]:
    # 打印用户问题
    print_header("User", question, "#00ff00")  # 亮绿色

    # 初始化历史
    history = {
        "messages": [HumanMessage(content=question + f" PAPER_META_INFO: {paper_info}")]
    }

    while True:
        try:
            # 显示 "思考中..." 提示
            loading_html = HTML(
                '<span style="color: #e6db74; font-style: italic;">🤖 AI 正在思考中...</span>'
            )
            display(loading_html)

            # 1. 调用 AI
            res = agent.invoke(history)
            raw_content = res["messages"][-1].content

            # 2. 【修复点】解析内容，确保它是字符串
            ai_content = parse_ai_response(raw_content)

            # 清除 "思考中" 的提示（这里我们简单地忽略，因为直接打印结果会把它顶上去）

            # 3. 打印 AI 回复
            print_header("AI Assistant", "", "#00ffff")  # 亮青色头
            print_ai_markdown(ai_content)  # Markdown 正文

            # 4. 获取下一次输入
            question = input("请输入你的问题 (exit 退出): ")

            if question.strip().lower() in ["exit", "quit"]:
                print("👋 已退出交互")
                break

            # 打印新一轮的用户问题
            print_header("User", question, "#00ff00")

            # 更新历史
            history["messages"] = res["messages"] + [HumanMessage(content=question)]

        except KeyboardInterrupt:
            print("\n🛑 用户强制中断")
            break
        except Exception as e:
            # 打印详细错误，方便调试
            print(f"❌ 发生错误: {e}")
            # 如果发生错误，打印一下原始数据类型，方便排查
            if "raw_content" in locals():
                print(f"调试信息: 返回的数据类型是 {type(raw_content)}")
            break
else:
    print("👋 已退出")


🟢 交互开始 (输入 'exit' 结束)


你好！很高兴为您介绍这篇论文《GLU Variants Improve Transformer》。

这篇论文由谷歌的Noam Shazeer于2020年发表，主要探讨了在Transformer模型中，使用门控线性单元（Gated Linear Units, GLU）变体替换标准ReLU或GELU激活函数的可能性，并发现某些变体能够提升模型的性能。

请问您对这篇论文有什么想了解的吗？我们可以一起探讨它的主要发现、实验细节或者任何您感兴趣的部分。

好的，我们来看一下这篇论文的实验结果。

### 实验设置

为了评估不同GLU变体的效果，作者在**Transformer**模型上进行了实验，这是一种广泛用于序列到序列任务的神经网络架构。实验主要集中在**机器翻译**任务上，具体使用的是**WMT 2014英语到法语**和**英语到德语**的数据集。

实验中，作者将Transformer模型前馈网络子层（feed-forward sublayers）中常用的**ReLU**或**GELU**激活函数，替换为不同的GLU变体。

### 实验结果与结论

实验结果表明，使用某些GLU变体可以显著提升翻译质量，效果优于传统的ReLU和GELU。以下是主要的发现：

1.  **GLU变体普遍优于ReLU**： 论文中的**表1**和**表2**（您可以在[原始论文的PDF](https://arxiv.org/pdf/2002.05202)中找到这些表格）清晰地展示了，几乎所有测试的GLU变体在翻译任务上的表现都超过了使用ReLU激活函数的基线模型。

2.  **最佳变体**： 在所有测试的变体中，**Bilinear**和**ReGLU**在英语到德语的任务上取得了最好的结果，而**GEGLU**在英语到法语的任务上表现最佳。

3.  **参数效率**： 尽管GLU及其变体在前馈网络层中引入了更多的参数，但实验发现，即使在总参数数量与基线ReLU模型大致相同的情况下，这些变体依然能够取得更好的性能。这说明GLU变体的优势不仅仅来自于参数数量的增加。

4.  **不同非线性函数的尝试**： 论文还探索了使用不同非线性函数（如Swish、GELU、ELU等）来替代GLU中的Sigmoid函数。结果显示，这些变体同样能够带来性能提升。

总的来说，这篇论文的实验结果有力地证明了，在Transformer模型中使用GLU变体是一个非常有效的改进方向，能够带来模型性能的提升。这一发现也对后续的大型语言模型（如T5、PaLM等）产生了深远的影响，许多模型都采纳了类似的结构。

👋 已退出交互
